IST - DBM2

# Data mining Project: : Titanic disaster Analysis

## Chad Long & Francesca Costa





In this project we'd like to analyse a dataset about the Titanic disaster with some methods studied in class.
Let's start presenting the data

In [513]:
import pandas as pd
import csv
%matplotlib

Using matplotlib backend: MacOSX


In [465]:
data = pd.read_csv('/Users/Francesca/Desktop/titanic1/train.csv')
data.columns = ['PassengerID', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 
               'Ticket', 'Fare', 'Cabin', 'Embarked']

data.head(10)


,PassengerID,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


First of all, we want to check if there're some duplicated rows (and delete them in case).

In [467]:
dups = data.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))

Number of duplicate rows = 0


There are no duplicated rows, so we can procede.


We wanted to look in more details the numbers of tuples and attributes and the number of missing values.

In [468]:
import numpy as np

print('Number of instances = %d' % data.shape[0])
print('Number of attributes = %d' % data.shape[1])

print('Number of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col, data[col].isna().sum()))

Number of instances = 891
Number of attributes = 12
Number of missing values:
	PassengerID: 0
	Survived: 0
	Pclass: 0
	Name: 0
	Sex: 0
	Age: 177
	SibSp: 0
	Parch: 0
	Ticket: 0
	Fare: 0
	Cabin: 687
	Embarked: 2


As we can see, in our dataset there are 891 istances and 12 attributes. However, not all columns are really needed for our analysis and we want to drop them.
In this first step we will delete:
- PassegnerID
- Name
- Ticket

In [469]:
dataShort = data.drop(['PassengerID','Name', 'Ticket'],axis=1)

dataShort.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


We noticed that 3 columns present missing some missing values:
- Age
- Cabin
- Embarked

In [470]:
total = dataShort.isnull().sum().sort_values(ascending=False)
percent_1 = dataShort.isnull().sum()/dataShort.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(3)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2


The column 'Cabin' presents lots of missing values (77% of the total). We decided to drop it.

In [471]:
dataShort = dataShort.drop(['Cabin'],axis=1)

dataShort.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Then, in order to fill 'Age' missed values, we decided to replace NaN with the female average age for female passengers and the male average age for male passengers (rounded).

In [472]:
women = dataShort[dataShort['Sex']=='female']
women_avgAge = round(women["Age"].mean())
print('Women average age = %d' % women_avgAge)

men = dataShort[dataShort['Sex']=='male']
men_avgAge = round(men["Age"].mean())
print('Men average age = %d' % men_avgAge)

Women average age = 28
Women average age = 31


Se ci fosse stato un flag solo sulla presenza dei genitori o bambini, si sarebbe potuto utilizzare per stimare l'età dei dati mancanti

In [473]:
dataShort.loc[(dataShort.Sex =='female') & (dataShort.Age.isnull()), 'Age'] = women_avgAge
dataShort.loc[(dataShort.Sex == 'male') & (dataShort.Age.isnull()), 'Age'] = men_avgAge

dataShort.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


The column 'Embarked' has only 2 missing values. Counting the different values we obtain:

In [474]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

We decided to fill those missing two values with the most probable value S.

In [476]:
dataShort.loc[dataShort.Embarked.isnull(), 'Embarked'] = 'S'

Let's check the number of missing values now:

In [477]:
print('Number of instances = %d' % dataShort.shape[0])
print('Number of attributes = %d' % dataShort.shape[1])

print('Number of missing values:')
for col in dataShort.columns:
    print('\t%s: %d' % (col, dataShort[col].isna().sum()))

Number of instances = 891
Number of attributes = 8
Number of missing values:
	Survived: 0
	Pclass: 0
	Sex: 0
	Age: 0
	SibSp: 0
	Parch: 0
	Fare: 0
	Embarked: 0


In [512]:
dataShort.to_csv('cleaned_data.csv', index = False)

Now that we've cleaned and tidied our dataset, let's procede in our analysis

## Frequent pattern mining

In [520]:
sns.set(style="whitegrid")
ax = sns.countplot(y='Pclass', hue="Survived", data=dataShort)
plt.ylabel('Pclass')
plt.title('Survival Plot')
plt.show()

import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))
women = dataShort[dataShort['Sex']=='female']
men = dataShort[dataShort['Sex']=='male']
ax = sns.histplot(women[women['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[0], kde =False)
ax = sns.histplot(women[women['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[0], kde =False)
ax.legend()
ax.set_title('Female')
ax = sns.histplot(men[men['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[1], kde = False)
ax = sns.histplot(men[men['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[1], kde = False)
ax.legend()
_ = ax.set_title('Male')

dataShort['Sex'].value_counts()

sns.barplot(x='Pclass', y='Survived', data=dataShort)

We want to show the correlation between the first class and the survival

In [506]:
firstClass_survived = ((dataShort['Pclass'] == 1) & (dataShort['Survived']== 1)).sum()
firstClass_notSurvived = ((dataShort['Pclass'] == 1) & (dataShort['Survived']== 0)).sum()
otherClass_survived = ((dataShort['Pclass'] != 1) & (dataShort['Survived'] == 1)).sum()
otherClass_notSurvived = ((dataShort['Pclass'] != 1) & (dataShort['Survived'] == 0)).sum()

firstClass_total = (dataShort['Pclass'] == 1).sum()
otherClass_total = (dataShort['Pclass'] != 1).sum()
survived_total = (dataShort['Survived'] == 1).sum()
notSurvived_total = (dataShort['Survived'] == 0).sum()
total = survived_total + notSurvived_total

print('First class survived: ', firstClass_survived)
print('First class not survived: ', firstClass_notSurvived)
print('Other classes survived: ', otherClass_survived)
print('Other classes not survived: ', otherClass_notSurvived)
print('\nFirst class total: ', firstClass_total)
print('Other classes total: ', otherClass_total)
print('Survived total: ', survived_total)
print('Not survived total: ', notSurvived_total)
print('\nTotal sum: ', total)

First class survived:  136
First class not survived:  80
Other classes survived:  206
Other classes not survived:  469

First class total:  216
Other classes total:  675
Survived total:  342
Not survived total:  549

Total sum:  891


We computed the correlation matrix to study the relationship between all the variables

In [523]:
cdata = dataShort


rep = {"male": 1, "female": 2}
cdata.replace({'Sex' : rep}, inplace=True)

rep = {"S": 1, "C": 2, 'Q':3}
cdata.replace({'Embarked' : rep}, inplace=True)

cdata.head()


corr_df = cdata.corr()
print("The correlation DataFrame is: \n")
print(corr_df, "\n")

The correlation DataFrame is: 

          Survived    Pclass       Sex       Age     SibSp     Parch  \
Survived  1.000000 -0.338481  0.543351 -0.081736 -0.035322  0.081629   
Pclass   -0.338481  1.000000 -0.131900 -0.329201  0.083081  0.018443   
Sex       0.543351 -0.131900  1.000000 -0.104850  0.114631  0.245489   
Age      -0.081736 -0.329201 -0.104850  1.000000 -0.237059 -0.183536   
SibSp    -0.035322  0.083081  0.114631 -0.237059  1.000000  0.414838   
Parch     0.081629  0.018443  0.245489 -0.183536  0.414838  1.000000   
Fare      0.257307 -0.549500  0.182333  0.088270  0.159651  0.216225   
Embarked  0.106811  0.045702  0.116569  0.003538 -0.059961 -0.078665   

              Fare  Embarked  
Survived  0.257307  0.106811  
Pclass   -0.549500  0.045702  
Sex       0.182333  0.116569  
Age       0.088270  0.003538  
SibSp     0.159651 -0.059961  
Parch     0.216225 -0.078665  
Fare      1.000000  0.062142  
Embarked  0.062142  1.000000   



From the correlation Matrix we can notice that sex and survival are highly correlated.

In [524]:
sns.set(style="whitegrid")
ax = sns.countplot(y='Sex', hue="Survived", data=dataShort)
plt.ylabel('Sex')
plt.title('Survival Plot')
plt.show()

Trovare la classe in base a sesso, età e famiglia

Trovare la correlazione tra età e tipo di parenti
Correlare il prezzo del biglietto con la classe o il porto di imbarco

Albero decisionale: fai diverse prove con diversi alberi decisionali con diverse feature in input per evitare l'overfitting
Non testare l'argoritmo solo su un dataset completo, ma provare varie features per verificare l'overfitting
Valutare la bontà dell'algoritmo con tutti i parametri della classificazione.
Valutare le metriche sempre sul test set
overfitting = all'algoritmo passi troppi parametri e l'argoritmo impara a memoria.
Non identifica l'andamento generico dei dati. Non è rappresentativo della varietà dei dati, ma solo dei dati già osservati
Accuratezza non è fiducia
- vuoi evitare falsi negative
- sbilanciato
Considerare anche recall e la precision
Modello stupido che considera solo la possibilità più alta.

Capitolo 2-5-8.1(8.2, 8.3 no) Random forest
Grafico rispetto alla profondità del grafico e valutare l'overfitting

## Classification